In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.1'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [624 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,277 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,430 kB]
Get:13 http://archive

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df=spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home_sales')

In [14]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

avg_4house_price=spark.sql("SELECT ROUND(AVG(price),2), YEAR(date) from home_sales WHERE bedrooms==4 GROUP BY YEAR(date) ORDER BY YEAR(date)")
avg_4house_price.show()

+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|            300263.7|      2019|
|           298353.78|      2020|
|           301819.44|      2021|
|           296363.88|      2022|
+--------------------+----------+



In [17]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

avg_price_3bed_3bath=spark.sql("SELECT ROUND(AVG(price),2), date_built from home_sales WHERE bedrooms==3 AND bathrooms==3 GROUP BY date_built ORDER BY date_built")
avg_price_3bed_3bath.show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           292859.62|      2010|
|           291117.47|      2011|
|           293683.19|      2012|
|           295962.27|      2013|
|           290852.27|      2014|
|            288770.3|      2015|
|           290555.07|      2016|
|           292676.79|      2017|
+--------------------+----------+



In [18]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

avg_price_2floors=spark.sql("SELECT ROUND(AVG(price),2), date_built from home_sales WHERE bedrooms==3 AND bathrooms==3 AND floors==2 AND sqft_living>=2000 GROUP BY date_built ORDER BY date_built")
avg_price_2floors.show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           285010.22|      2010|
|           276553.81|      2011|
|           307539.97|      2012|
|           303676.79|      2013|
|           298264.72|      2014|
|           297609.97|      2015|
|            293965.1|      2016|
|           280317.58|      2017|
+--------------------+----------+



In [19]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from home_sales GROUP BY view HAVING ROUND(AVG(price),2)>=35000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           288929.09|
|  51|           788128.21|
|  15|           284907.04|
|  54|           798684.82|
|  11|           280356.07|
|  29|           283881.72|
|  69|           750537.94|
|  42|           289225.45|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|   3|           284314.53|
|  30|           281085.62|
|  34|           286124.07|
|  59|            791453.0|
|   8|           279099.78|
|  22|           284908.42|
|  28|           285474.25|
|  85|          1056336.74|
|  16|           291990.83|
+----+--------------------+
only showing top 20 rows

--- 0.8960161209106445 seconds ---


In [20]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [21]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [22]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from home_sales GROUP BY view HAVING ROUND(AVG(price),2)>=35000").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           288929.09|
|  51|           788128.21|
|  15|           284907.04|
|  54|           798684.82|
|  11|           280356.07|
|  29|           283881.72|
|  69|           750537.94|
|  42|           289225.45|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|   3|           284314.53|
|  30|           281085.62|
|  34|           286124.07|
|  59|            791453.0|
|   8|           279099.78|
|  22|           284908.42|
|  28|           285474.25|
|  85|          1056336.74|
|  16|           291990.83|
+----+--------------------+
only showing top 20 rows

--- 0.6204798221588135 seconds ---


In [23]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.parquet('home_parquet',mode='overwrite')

In [24]:
# 11. Read the parquet formatted data.
df_parquet=spark.read.parquet('home_parquet')

In [26]:
# 12. Create a temporary table for the parquet data.
df_parquet.createOrReplaceTempView('temp_sales_parquet')

In [27]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from temp_sales_parquet GROUP BY view HAVING ROUND(AVG(price),2)>=35000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           288929.09|
|  51|           788128.21|
|  15|           284907.04|
|  54|           798684.82|
|  11|           280356.07|
|  29|           283881.72|
|  69|           750537.94|
|  42|           289225.45|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|   3|           284314.53|
|  30|           281085.62|
|  34|           286124.07|
|  59|            791453.0|
|   8|           279099.78|
|  22|           284908.42|
|  28|           285474.25|
|  85|          1056336.74|
|  16|           291990.83|
+----+--------------------+
only showing top 20 rows

--- 0.7893688678741455 seconds ---


In [28]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [29]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False